# Coordonnées géographiques

## 1.Traitement des données

### Importation

In [1]:
import numpy as np
import pandas as pd

# Importation des fonctions dont l'on se servira pour toutes les figures
from bokeh.plotting import figure, output_notebook, show

# Précision de l'affichage des graphiques dans des cellules jupyter
output_notebook()

Loading BokehJS ...

In [2]:
# Définition du chemin d'accès:
### Céline
#%cd C:\Users\c.doussot\Desktop\Data Analyst\GitHub\ParisPyVelib_Datas
### Hermine
%cd C:\Users\h.berthon\Documents\GitHub\ParisPyVelib\ParisPyVelib_Datas
### Tarik
#%cd C:\Users\Home\Documents\Git\ParisPyVelib\Data

df = pd.read_csv('2018-2021_comptage-velo-donnees-compteurs.csv')

C:\Users\h.berthon\Desktop\Data Sc\Projet Velib


### Nouveau DataFrame

In [3]:
#Nouveau df avec moins de colonnes
df_lat_lon= df.drop(['Unnamed: 0','Id_old','Address_old','Date_count', 'Date_instal', 'Photo_old', 
                      'Coord', 'Latitude', 'Longitude', 'Direction', 'Y_Date_Count', 'M_Date_Count', 'D_Date_Count',
                     'Dweek_Date_Count', 'H_Date_Count', 'Y_Date_Instal', 'M_Date_Instal', 'D_Date_Instal'
                     ], axis =1)

df_lat_lon.head()

,Id,Address,Count_by_hour,Coord_old,Source
0,100003096,97 avenue Denfert Rochereau,0.0,"48.83511,2.33338",2021
1,100003096,97 avenue Denfert Rochereau,0.0,"48.83511,2.33338",2021
2,100003096,97 avenue Denfert Rochereau,0.0,"48.83511,2.33338",2021
3,100003096,97 avenue Denfert Rochereau,3.0,"48.83511,2.33338",2021
4,100003096,97 avenue Denfert Rochereau,7.0,"48.83511,2.33338",2021


In [4]:
#Changement de format de coordonnées géo

# Création des colonnes 'longitude', et 'latitude'
df_lat_lon[['Latitude', 'Longitude']] = df_lat_lon['Coord_old'].str.split(',', expand = True)

#Les coordonnées que nous avons sont au format Degrés décimaux (DD) = 5 décimales
df_lat_lon['Latitude'] = df_lat_lon['Latitude'].astype('float64').round(5)
df_lat_lon['Longitude'] = df_lat_lon['Longitude'].astype('float64').round(5)

#Nouvelle colonne avec les coordonnées arrondies
df_lat_lon['Coord'] = df_lat_lon['Latitude'].astype('str') + ',' + df_lat_lon['Longitude'].astype('str')

#Suppression colonnes inutiles
df_lat_lon = df_lat_lon.drop(['Coord_old', 'Latitude', 'Longitude'], axis=1)

#Adresse nom du site de comptage en majuscule
df_lat_lon['Address'] = df_lat_lon['Address'].apply(lambda x : x.upper())

#Tri par adresse
df_lat_lon = df_lat_lon.sort_values(by = ['Address'], axis=0)

df_lat_lon.head()

,Id,Address,Count_by_hour,Source,Coord
1712536,100044494,10 AVENUE DE LA GRANDE ARMÉE SE-NO,14.0,2018,"48.87472,2.29244"
1565009,100044494,10 AVENUE DE LA GRANDE ARMÉE SE-NO,11.0,2019,"48.87472,2.29244"
1565008,100044494,10 AVENUE DE LA GRANDE ARMÉE SE-NO,3.0,2019,"48.87472,2.29244"
1565007,100044494,10 AVENUE DE LA GRANDE ARMÉE SE-NO,17.0,2019,"48.87472,2.29244"
1565006,100044494,10 AVENUE DE LA GRANDE ARMÉE SE-NO,2.0,2019,"48.87472,2.29244"


In [5]:
#Nombre de valeurs uniques avant le traitement des données
df_lat_lon.nunique()

Id                 71
Address            81
Count_by_hour    1054
Source              4
Coord              84
dtype: int64

Pour pouvoir représenter les différentes addresses sur une carte intéractive, il faut que les addresses correspondent aux coordonnées

### Traitement des addresses et coordonnées géographiques

Je n'ai pas réussi à faire de programme pour remplacer les données, j'ai donc fait le tout manuellement

In [6]:
df_lat_lon[['Id','Coord','Address', 'Source']].groupby(['Address', 'Id','Coord', 'Source']).count().head(15)

Empty DataFrame
Columns: []
Index: [(10  AVENUE DE LA GRANDE ARMÉE SE-NO, 100044494, 48.87472,2.29244, 2018), (10  AVENUE DE LA GRANDE ARMÉE SE-NO, 100044494, 48.87472,2.29244, 2019), (10 AVENUE DE LA GRANDE ARMÉE SE-NO, 100044494, 48.87472,2.29244, 2020), (10 AVENUE DE LA GRANDE ARMÉE SE-NO, 100044494, 48.87472,2.29244, 2021), (10 BD AUGUSTE BLANQUI NE-SO, 100049408, 48.8309,2.35324, 2018), (10 BD AUGUSTE BLANQUI NE-SO, 100049408, 48.8309,2.35324, 2019), (10 BOULEVARD AUGUSTE BLANQUI NE-SO, 100049408, 48.83068,2.35348, 2020), (10 BOULEVARD AUGUSTE BLANQUI NE-SO, 100049408, 48.83068,2.35348, 2021), (100 RUE LA FAYETTE, 100003099, 48.87746,2.35008, 2020), (100 RUE LA FAYETTE, 100003099, 48.87746,2.35008, 2021), (100 RUE LA FAYETTE O-E, 100003099, 48.87746,2.35008, 2018), (100 RUE LA FAYETTE O-E, 100003099, 48.87746,2.35008, 2019), (102 BD MAGENTA SE-NO, 100047536, 48.87773,2.35493, 2018), (102 BD MAGENTA SE-NO, 100047536, 48.87773,2.35493, 2019), (102 BOULEVARD DE MAGENTA SE-NO, 100047536, 48.87773,2.35493, 2020)]

On remarque que la notation de certaines addresses a évolué en 2020, de même pour les coordonnées géographiques, pour simplifier je vais modifier les addresses et coordonnées géographiques en prenant la valeur récente de 2021

In [7]:
#Groupby pour voir de manière globale les différences entre addresses et coordonnées géographiques
df_lat_lon[['Address','Id','Coord',]].groupby(['Coord','Id','Address']).count().head(60)

,,
Coord,Id,Address
"48.82024,2.35902",100056039,180 AVENUE D'ITALIE N-S
"48.82026,2.3592",100056041,147 AVENUE D'ITALIE S-N
"48.82108,2.32537",100056225,3 AVENUE DE LA PORTE D'ORLÉANS S-N
"48.82203,2.32545",100056225,3 AVENUE DE LA PORTE D'ORLÉANS S-N
"48.82636,2.30303",100047547,6 RUE JULIA BARTET
"48.82648,2.30315",100047547,6 RUE JULIA BARTET
"48.8265,2.38434",100047541,PONT NATIONAL SO-NE
"48.82658,2.38409",100047541,PONT NATIONAL SO-NE
"48.82682,2.38465",100047549,PONT NATIONAL NE-SO


In [8]:
#Vérification des cohérences entre adresse, Id et coordonnées géographiques
df_inter = df_lat_lon[(df_lat_lon['Id'] == 100065336)| (df_lat_lon['Id'] == 100047550)]
df_inter[['Id','Coord','Address', 'Source']].groupby([ 'Id','Address','Coord', 'Source']).count()

Empty DataFrame
Columns: []
Index: [(100047550, 35 BOULEVARD DE MENILMONTANT NO-SE, 48.86053,2.38836, 2018), (100047550, 35 BOULEVARD DE MENILMONTANT NO-SE, 48.86053,2.38836, 2019), (100065336, 35 BOULEVARD DE MÉNILMONTANT, 48.86053,2.38836, 2021)]

En utilisant le groupby on peut identifier de manière plus précise les Id posant problème et choisir la donnée la plus récente

In [9]:
#Certaines adresses présentent des espaces qui sont gommées à l'affichage du groupby
for i in df_lat_lon['Address'].unique():
    print(i)

10  AVENUE DE LA GRANDE ARMÉE SE-NO
10 AVENUE DE LA GRANDE ARMÉE SE-NO
10 BD AUGUSTE BLANQUI NE-SO
10 BOULEVARD AUGUSTE BLANQUI NE-SO
100 RUE LA FAYETTE
100 RUE LA FAYETTE O-E
102 BD MAGENTA SE-NO
102 BOULEVARD DE MAGENTA SE-NO
105 RUE LA FAYETTE E-O
106 AVENUE DENFERT ROCHEREAU NE-SO
129 RUE LECOURBE SO-NE
132 RUE LECOURBE NE-SO
135 AVENUE DAUMESNIL SE-NO
147 AVENUE D'ITALIE S-N
152 BOULEVARD DU MONTPARNASSE
16 AVENUE DE LA PORTE DES TERNES E-O
18 QUAI DE L'HOTEL DE VILLE
18 QUAI DE L'HÔTEL DE VILLE
180 AVENUE D'ITALIE N-S
2 AVENUE DE LA PORTE DE BAGNOLET O-E
20 AVENUE DE CLICHY
21 BOULEVARD AUGUSTE BLANQUI SO-NE
21 BOULEVARD SAINT MICHEL S-N
243 BOULEVARD SAINT GERMAIN NO-SE
254 RUE DE VAUGIRARD
26 BOULEVARD DE MÉNILMONTANT SE-NO
27 BOULEVARD DIDEROT E-O
27 QUAI DE LA TOURNELLE
28 BOULEVARD DIDEROT
3 AVENUE DE LA PORTE D'ORLÉANS S-N
30 RUE SAINT JACQUES N-S
33 AVENUE DES CHAMPS ELYSÉES NO-SE
35 BOULEVARD DE MENILMONTANT NO-SE
35 BOULEVARD DE MÉNILMONTANT
36 QUAI DE GRENELLE
38 RUE TU

In [10]:
#Remplacement au niveau des adresses
df_lat_lon['Address'] = df_lat_lon['Address'].replace({'10  AVENUE DE LA GRANDE ARMÉE SE-NO' : '10 AVENUE DE LA GRANDE ARMÉE',
                                                      '10 AVENUE DE LA GRANDE ARMÉE SE-NO' : '10 AVENUE DE LA GRANDE ARMÉE',
                                                      '100 RUE LA FAYETTE O-E' : '100 RUE LA FAYETTE',
                                                      '102 BOULEVARD DE MAGENTA SE-NO':'102 BD MAGENTA SE-NO',
                                                      '10 BOULEVARD AUGUSTE BLANQUI NE-SO' : '10 BD AUGUSTE BLANQUI NE-SO',
                                                      '72 AVENUE FLANDRE SO-NE':'72 AVENUE DE FLANDRE SO-NE',
                                                      '97 AVENUE DENFERT ROCHEREAU SO-NE':'97 AVENUE DENFERT ROCHEREAU',
                                                      "18 QUAI DE L'HOTEL DE VILLE": "18 QUAI DE L'HÔTEL DE VILLE",
                                                      'PONT DU GARIGLIANO SE-NO':'PONT DU GARIGLIANO',
                                                       '77 BOULEVARD RICHARD LENOIR N-S':'77 BD RICHARD LENOIR N-S',
                                                       '35 BOULEVARD DE MENILMONTANT NO-SE':'35 BOULEVARD DE MÉNILMONTANT',
                                                       '72 BOULEVARD RICHARD LENOIR  S-N':'72 BD RICHARD LENOIR  S-N'
                                                       
                                                      })

#Remplacement au niveau des Lat & Long
df_lat_lon['Coord'] = df_lat_lon['Coord'].replace({'48.8309,2.35324' : '48.83068,2.35348',
                                                   '48.83514,2.33303' : '48.83521,2.33307',
                                                  '48.86062,2.38872' : '48.86057,2.38886',
                                                  '48.82203,2.32545' : '48.82108,2.32537',
                                                  '48.85204,2.28589' : '48.85209,2.28508',
                                                  '48.8344,2.37694':'48.83436,2.377',
                                                  '48.82648,2.30315': '48.82636,2.30303',
                                                  '48.8484,2.27593':'48.8484,2.27586',
                                                 '48.8347,2.33297':'48.83511,2.33338',
                                                  '48.83993,2.26715':'48.84015,2.26733',
                                                   '48.8265,2.38434':'48.82658,2.38409',
                                                   '48.83988,2.2671':'48.83992,2.26694',
                                                   '48.86077,2.37274':'48.86077,2.37305',
                                                   '48.89141,2.38495':'48.89142,2.38495'
                                                  })
#Remplacement identifiant
df_lat_lon['Id'] = df_lat_lon['Id'].replace({100047550:100065336})

In [11]:
df_lat_lon.nunique()

Id                 70
Address            70
Count_by_hour    1054
Source              4
Coord              70
dtype: int64

Les identifiant, addresses et coordonnées géographiques sont les mêmes pour toutes les données provenant de 2018 à 2021

On peut maintenant séparer les coordonnées géographiques

In [12]:
df_lat_lon[['Latitude', 'Longitude']] = df_lat_lon['Coord'].str.split(',', expand = True)

#Suppression colonnes inutiles
df_lat_lon = df_lat_lon.drop(['Coord', 'Source'], axis=1)

df_lat_lon.head()

,Id,Address,Count_by_hour,Latitude,Longitude
1712536,100044494,10 AVENUE DE LA GRANDE ARMÉE,14.0,48.87472,2.29244
1565009,100044494,10 AVENUE DE LA GRANDE ARMÉE,11.0,48.87472,2.29244
1565008,100044494,10 AVENUE DE LA GRANDE ARMÉE,3.0,48.87472,2.29244
1565007,100044494,10 AVENUE DE LA GRANDE ARMÉE,17.0,48.87472,2.29244
1565006,100044494,10 AVENUE DE LA GRANDE ARMÉE,2.0,48.87472,2.29244


## 2. Mapping

In [13]:
#Il faut changer coord en mercator
import math

# derived from the Java version explained here: http://wiki.openstreetmap.org/wiki/Mercator
RADIUS = 6378137.0 # in meters on the equator

def lat2y(a):
    return math.log(math.tan(math.pi / 4 + math.radians(a) / 2)) * RADIUS

def lon2x(a):
    return math.radians(a) * RADIUS

In [14]:
df_lat_lon.info()
#Changement de type de Latitude et Longitude

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1723879 entries, 1712536 to 1668798
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Id             int64  
 1   Address        object 
 2   Count_by_hour  float64
 3   Latitude       object 
 4   Longitude      object 
dtypes: float64(1), int64(1), object(3)
memory usage: 78.9+ MB


In [15]:
df_lat_lon['Latitude'] = df_lat_lon['Latitude'].astype('float64')
df_lat_lon['Longitude'] = df_lat_lon['Longitude'].astype('float64')

In [16]:
#Vérification
df_lat_lon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1723879 entries, 1712536 to 1668798
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Id             int64  
 1   Address        object 
 2   Count_by_hour  float64
 3   Latitude       float64
 4   Longitude      float64
dtypes: float64(3), int64(1), object(1)
memory usage: 78.9+ MB


In [17]:
df_lat_lon['Latitude M'] = df_lat_lon['Latitude'].apply(lat2y)
df_lat_lon['Longitude M'] = df_lat_lon['Longitude'].apply(lon2x)

In [18]:
df_lat_lon.head()

,Id,Address,Count_by_hour,Latitude,Longitude,Latitude M,Longitude M
1712536,100044494,10 AVENUE DE LA GRANDE ARMÉE,14.0,48.87472,2.29244,6.253631e+06,255193.253474
1565009,100044494,10 AVENUE DE LA GRANDE ARMÉE,11.0,48.87472,2.29244,6.253631e+06,255193.253474
1565008,100044494,10 AVENUE DE LA GRANDE ARMÉE,3.0,48.87472,2.29244,6.253631e+06,255193.253474
1565007,100044494,10 AVENUE DE LA GRANDE ARMÉE,17.0,48.87472,2.29244,6.253631e+06,255193.253474
1565006,100044494,10 AVENUE DE LA GRANDE ARMÉE,2.0,48.87472,2.29244,6.253631e+06,255193.253474


In [19]:
print('Latitude min : ', df_lat_lon['Latitude M'].min())
print('Latitude max : ', df_lat_lon['Latitude M'].max())
print('Longitude min : ', df_lat_lon['Longitude M'].min())
print('Longitude max : ', df_lat_lon['Longitude M'].max())

Latitude min :  6244414.743721515
Latitude max :  6257395.712802779
Longitude min :  252185.40083289784
Longitude max :  268245.4637696434


In [20]:
#On groupe le dataFrame pour avoir le nombre de comptage total par adresse
df2 = df_lat_lon.groupby(['Address','Latitude', 'Longitude', 'Latitude M', 'Longitude M']).agg({'Count_by_hour' : lambda Count_by_hour : Count_by_hour.count()}).reset_index()
df2 = df2.sort_values(by = ['Count_by_hour'],ascending=False, axis=0)
df2.head()

,Address,Latitude,Longitude,Latitude M,Longitude M,Count_by_hour
12,18 QUAI DE L'HÔTEL DE VILLE,48.85372,2.35702,6.250077e+06,262382.266190,59664.0
30,39 QUAI FRANÇOIS MAURIAC,48.83436,2.37700,6.246802e+06,264606.429616,59663.0
23,28 BOULEVARD DIDEROT,48.84603,2.37543,6.248776e+06,264431.658015,59663.0
69,VOIE GEORGES POMPIDOU,48.84840,2.27586,6.249177e+06,253347.576317,58753.0
48,FACE AU 25 QUAI DE L'OISE,48.89142,2.38495,6.256458e+06,265491.419567,43772.0


In [21]:
#Top 10 des addresses avec le plus passage total
df3 = df2.head(10)
df3

,Address,Latitude,Longitude,Latitude M,Longitude M,Count_by_hour
12,18 QUAI DE L'HÔTEL DE VILLE,48.85372,2.35702,6.250077e+06,262382.266190,59664.0
30,39 QUAI FRANÇOIS MAURIAC,48.83436,2.37700,6.246802e+06,264606.429616,59663.0
23,28 BOULEVARD DIDEROT,48.84603,2.37543,6.248776e+06,264431.658015,59663.0
69,VOIE GEORGES POMPIDOU,48.84840,2.27586,6.249177e+06,253347.576317,58753.0
48,FACE AU 25 QUAI DE L'OISE,48.89142,2.38495,6.256458e+06,265491.419567,43772.0
32,6 RUE JULIA BARTET,48.82636,2.30303,6.245450e+06,256372.126882,43771.0
51,FACE AU 48 QUAI DE LA MARNE,48.89122,2.38573,6.256424e+06,265578.248770,43724.0
46,FACE 104 RUE D'AUBERVILLIERS,48.89046,2.36885,6.256295e+06,263699.175766,43723.0
52,FACE AU 70 QUAI DE BERCY,48.82952,2.38699,6.245984e+06,265718.511329,43534.0
10,152 BOULEVARD DU MONTPARNASSE,48.84080,2.33323,6.247892e+06,259733.975504,43328.0


In [22]:
from bokeh.tile_providers import get_provider
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models.tools import HoverTool

source = ColumnDataSource(df2)
source2 = ColumnDataSource(df3)

hover = HoverTool(tooltips=[("Address", "@Address"), ("Lat", "@Latitude"), ("Lng", "@Longitude")])

p = figure(x_range = (250000, 270000), y_range = (6244414, 6257395), 
           x_axis_type = 'mercator', y_axis_type = 'mercator')
tuile = get_provider('OSM')
p.add_tile(tuile)
p.circle(x='Longitude M', y='Latitude M',source = source, 
         fill_color = 'navy',line_color='grey', size = 10)
p.circle(x='Longitude M', y='Latitude M',source = source2, 
         fill_color = 'red',line_color='grey', size = 10)
p.add_tools(hover)

show(p)

In [23]:
#attention top10 peut dépendre de count_by_hour bornes plus vieilles
#séparer traitement de données et bokeh
